In [ ]:
import numpy as np


class InputSet:
    def __init__(self, height, width, ):
        self.event_bitrate = None
        self.adder_features = np.zeros((height, width), dtype=bool)
        self.adder_time = None
        self.cv_features = np.zeros((height, width), dtype=bool)
        self.cv_time = None
        self.psnr = None
        self.mse = None
        self.ssim = None
        self.precision = None
        self.recall = None
        self.accuracy = None

    def total_adder(self):
        return np.sum(self.adder_features.astype(int))

    def total_cv(self):
        return np.sum(self.cv_features.astype(int))
    

In [ ]:
def process_entry(entry, input_sets):
    if 's' in entry:
        if "OpenCV" in entry['s']:
            input_sets[-1].cv_features[entry['y'], entry['x']] = True
        elif "ADDER" in entry['s']:
            input_sets[-1].adder_features[entry['y'], entry['x']] = True
        else:
            raise Exception("Malformed entry")
    elif 'psnr' in entry:
        input_sets[-1].psnr = entry['psnr']
        input_sets[-1].mse = entry['mse']
        input_sets[-1].ssim = entry['ssim']


In [ ]:
import pickle

objects = []
with (
        open(
            "/home/andrew/Code/adder-codec-rs/adder-codec-rs/9_raw_VIRAT_S_050201_05_000890_000944__31_10_2023_13_37_00.log",
            "rb")) as openfile:
    # Read the dimensions
    dims = openfile.readline().decode('UTF-8')
    print(dims)
    width = int(dims.split('x')[0])
    height = int(dims.split('x')[1])
    channels = int(dims.split('x')[2])
    #
    dbg_lines = 0
    #
    input_interval_idx = 0
    #
    input_sets = [InputSet(height, width)]
    recon_sets = [InputSet(height, width)]

    while True:
        try:
            entry = pickle.load(openfile)
            if type(entry) == str and "ADDER FAST:" in entry:
                # The start of a new interval
                time = entry.split(': ')[1]
                input_sets[-1].adder_time = int(time)
                continue
            elif type(entry) == str and "bps:" in entry:
                # The bitrate
                # The end of the interval
                bitrate = float(entry.split(': ')[1]) / 1024.0 / 1024.0
                input_sets[-1].event_bitrate = bitrate

                input_sets += [InputSet(height, width)]
                input_interval_idx += 1
                continue
            elif type(entry) == str and "OpenCV FAST:" in entry:
                time = entry.split(': ')[1]
                input_sets[-1].cv_time = int(time)
                
            elif type(entry) == str and "Feature results:" in entry:
                entry = pickle.load(openfile)
                input_sets[-1].precision = entry
                entry = pickle.load(openfile)
                input_sets[-1].recall = entry
                entry = pickle.load(openfile)
                input_sets[-1].accuracy = entry

            elif type(entry) == str and "META:" in entry:
                print(entry)
            elif type(entry) == str and "RECONSTRUCTION" in entry:
                print("got recon")
                break
            else:
                process_entry(entry, input_sets)

                # print(entry)
                dbg_lines += 1
                # if dbg_lines == 100000:
                #     break
        except pickle.UnpicklingError:
            line = openfile.readline().decode('UTF-8')
            if "Ticks per second" in line:
                tps = int(line.split(': ')[2])
            elif "ticks per source interval" in line:
                tpf = int(line.split(': ')[2])
            print('hm')
            
            print(line)
        except EOFError:
            break
            
        # Read the reconstructed quality metrics
    while True:
        # break
        try:
            print('a')
            entry = pickle.load(openfile)
            print(entry)
            process_entry(entry, recon_sets)
            recon_sets += [InputSet(height, width)]
        except EOFError:
            print('eof')
            break

    # Remove the last item because it might have none values
    input_sets = input_sets[:-1]
    recon_sets = recon_sets[:-1]


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import mpld3

adder_feature_count = [obj.total_adder() for obj in input_sets]
cv_feature_count = [obj.total_cv() for obj in input_sets]

sns.set(style="whitegrid")
plt.figure(figsize=(8, 4))

sns.lineplot(x=range(len(input_sets) - 1), y=adder_feature_count[:-1], marker=None, label="ADDER")
sns.lineplot(x=range(len(input_sets) - 1), y=cv_feature_count[:-1], marker=None, label="OpenCV")
plt.ylabel("# features")
plt.xlabel("Input frame")
plt.title("Feature count over time")

plt.show()


average_precision = np.mean(np.nan_to_num(np.array([obj.precision for obj in input_sets]), nan=1.0))
print(average_precision)
average_recall = np.mean(np.array([obj.recall for obj in input_sets]))
print(average_recall)
average_accuracy = np.mean(np.array([obj.accuracy for obj in input_sets]))
print(average_accuracy)
# interactive_plot = mpld3.display()
# interactive_plot

In [ ]:
adder_times = [obj.adder_time for obj in input_sets]
cv_times = [obj.cv_time for obj in input_sets]

sns.set(style="whitegrid")
plt.figure(figsize=(8, 4))

sns.lineplot(x=range(len(input_sets)), y=adder_times, marker=None, label="ADDER")
sns.lineplot(x=range(len(input_sets)), y=cv_times, marker=None, label="OpenCV")
plt.ylabel("Time (ns)")
plt.xlabel("Input frame")
plt.yscale('log')
plt.title("Feature detection speed")

plt.show()
# interactive_plot = mpld3.display()
# interactive_plot


cv_mean = np.mean(np.array(cv_times))
adder_mean = np.mean(np.array(adder_times))
percent_change = (adder_mean / cv_mean) - 1
print('OpenCV mean:', cv_mean, '  Median:', np.median(np.array(cv_times)))
print('ADDER mean:', adder_mean, "({:.1f}%)".format(percent_change * 100), '  Median:',
      np.median(np.array(adder_times)))


# Bitrate over time

In [ ]:
adder_bitrates = [obj.event_bitrate for obj in input_sets]

total_adder_size = np.sum(adder_bitrates) / (tps / tpf)
print(total_adder_size, "Mb")

source_raw_Mbps = (width * height * channels * (tps / tpf) * 8.0) / 1024.0 / 1024.0
total_raw_size = (width * height * channels * len(adder_bitrates) * 8.0) / 1024.0 / 1024.0
print(total_raw_size, "Mb")
framed_bitrates = [source_raw_Mbps for obj in input_sets]

sns.set(style="whitegrid")
plt.figure(figsize=(8, 4))

sns.lineplot(x=range(len(input_sets)), y=adder_bitrates, marker=None, label="ADDER")
sns.lineplot(x=range(len(input_sets)), y=framed_bitrates, marker=None, label="Framed")
plt.ylabel("Raw bitrate (Mb/s)")
plt.xlabel("Input frame")
# plt.yscale('log')
plt.title("Bitrate")

plt.show()


In [ ]:
adder_psnr = [obj.psnr for obj in input_sets]
adder_ssim = [obj.ssim for obj in input_sets]

sns.set(style="whitegrid")
plt.figure(figsize=(8, 4))

sns.lineplot(x=range(len(input_sets)), y=adder_psnr, marker=None, label="PSNR")
sns.lineplot(x=range(len(input_sets)), y=adder_ssim, marker=None, label="SSIM")
plt.ylabel("Score")
plt.xlabel("Input frame")
# plt.yscale('log')
plt.title("Quality metrics")

plt.show()